<a href="https://colab.research.google.com/github/mshojaei77/Awesome-Fine-tuning/blob/main/LazyAxolotl_Jamba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🦎 LazyAxolotl - Jamba

> 🗣️ [Large Language Model Course](https://github.com/mlabonne/llm-course)

❤️ Created by [@maximelabonne](https://twitter.com/maximelabonne).

This version of [LazyAxolotl](https://colab.research.google.com/drive/1TsDKNo2riwVmU55gjuBgB1AXVtRRfRHW) is dedicated to fine-tuning [ai21labs/Jamba-v0.1](https://huggingface.co/ai21labs/Jamba-v0.1). It uses the following version of [lazyaxolotl.sh](https://gist.githubusercontent.com/mlabonne/e9f53fca97d9edb7aca0717770185438) and has successfully fine-tuned several models, including [mlabonne/Jambatypus-v0.1](https://huggingface.co/mlabonne/Jambatypus-v0.1).

*Thanks to [Wing Lian](https://github.com/winglian) for making Axolotl and for his quick implementation of Jamba.*

⚠️ I recommende using 2xA100 80GB or equivalent to fine-tune this model. With a single A100 80GB, I wasn't able to merge the adapter in FP16. If you want to still use a single GPU, it will merge in 8-bit precision instead.

In [ ]:
MODEL = "Jambatypus-v0.1"
yaml_config = f"""
base_model: ai21labs/Jamba-v0.1
trust_remote_code: true

load_in_8bit: false
load_in_4bit: true
strict: false

datasets:
  - path: chargoddard/Open-Platypus-Chat
    type: sharegpt
chat_template: chatml
dataset_prepared_path:
val_set_size: 0.01
output_dir: ./out

sequence_len: 4096
sample_packing: true
pad_to_sequence_len: true
eval_sample_packing: false

use_wandb: true
wandb_project: axolotl
wandb_entity:
wandb_watch:
wandb_name: {MODEL}
wandb_log_model:

adapter: qlora
lora_r: 32
lora_alpha: 64
lora_dropout: 0.05
lora_target_linear: true

low_cpu_mem_usage: true
gradient_accumulation_steps: 8
micro_batch_size: 1
num_epochs: 1
optimizer: adamw_bnb_8bit
adam_beta2: 0.95
adam_epsilon: 0.00001
max_grad_norm: 1.0
lr_scheduler: cosine
learning_rate: 0.0002

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false
early_stopping_patience:
resume_from_checkpoint:
local_rank:
logging_steps: 1
xformers_attention:
flash_attention: true

warmup_steps: 10
evals_per_epoch: 4
saves_per_epoch: 4
save_total_limit: 2
debug:
deepspeed:
weight_decay: 0.0
special_tokens:
"""

In [ ]:
try:
  import runpod
except ImportError:
  !pip install -qqq runpod --progress-bar off

import runpod
from runpod.error import QueryError
from google.colab import userdata, runtime
import requests
import json
import yaml
import requests
import time

def upload_gist(text, gist_name, gh_token, gist_description=""):
    gist_content = {
        "description": gist_description,
        "public": False,
        "files": {
            f"{gist_name}": {
                "content": text
            }
        },
    }

    # Headers for the request
    headers = {
        "Authorization": f"token {gh_token}",
        "Accept": "application/vnd.github.v3+json",
    }

    # Make the request
    response = requests.post(
        "https://api.github.com/gists", headers=headers, json=gist_content
    )

    if response.status_code == 201:
        gist_data = response.json()
        raw_url = gist_data['files'][gist_name]['raw_url']
        print(f"Uploaded Axolotl config as gist: {raw_url}")
        return raw_url
    else:
        print(
            f"Failed to upload gist. Status code: {response.status_code}. Response: {response.text}"
        )
        return None

def delete_gist(gh_token, gist_id):
    # Headers for the request
    headers = {
        "Authorization": f"token {gh_token}",
        "Accept": "application/vnd.github.v3+json",
    }

    # Make the request
    response = requests.delete(
        f"https://api.github.com/gists/{gist_id}", headers=headers
    )

    if response.status_code == 204:
        print("Gist has been deleted.")
        return True
    else:
        print(
            f"Failed to delete gist. Status code: {response.status_code}. Response: {response.text}"
        )
        return False

# @title ## Training parameters
GPU = "NVIDIA A100 80GB PCIe" # @param ["NVIDIA A100 80GB PCIe", "NVIDIA A100-SXM4-80GB", "NVIDIA A30", "NVIDIA A40", "NVIDIA GeForce RTX 3070", "NVIDIA GeForce RTX 3080", "NVIDIA GeForce RTX 3080 Ti", "NVIDIA GeForce RTX 3090", "NVIDIA GeForce RTX 3090 Ti", "NVIDIA GeForce RTX 4070 Ti", "NVIDIA GeForce RTX 4080", "NVIDIA GeForce RTX 4090", "NVIDIA H100 80GB HBM3", "NVIDIA H100 PCIe", "NVIDIA L4", "NVIDIA L40", "NVIDIA RTX 4000 Ada Generation", "NVIDIA RTX 4000 SFF Ada Generation", "NVIDIA RTX 5000 Ada Generation", "NVIDIA RTX 6000 Ada Generation", "NVIDIA RTX A2000", "NVIDIA RTX A4000", "NVIDIA RTX A4500", "NVIDIA RTX A5000", "NVIDIA RTX A6000", "Tesla V100-FHHL-16GB", "Tesla V100-PCIE-16GB", "Tesla V100-SXM2-16GB", "Tesla V100-SXM2-32GB"]
NUMBER_OF_GPUS = 2 # @param {type:"slider", min:1, max:8, step:1}
CONTAINER_DISK = 300 # @param {type:"slider", min:50, max:500, step:25}
CLOUD_TYPE = "SECURE" # @param ["COMMUNITY", "SECURE"]
SCRIPT = "https://gist.githubusercontent.com/mlabonne/a73b7d543f49b7ea2b52068414304414/raw" # @param {type:"string"}
ZERO = "Stage 2" # @param ["None", "Stage 2"]
LLM_AUTOEVAL = False # @param {type:"boolean"}
DEBUG = False # @param {type:"boolean"}

# @markdown ---

# @markdown ## Tokens
# @markdown Enter the name of your tokens in the Secrets tab.
USERNAME = "mlabonne" # @param {type:"string"}
RUNPOD_TOKEN = "runpod" # @param {type:"string"}
HUGGING_FACE_TOKEN = "HF_TOKEN" # @param {type:"string"}
WANDB_TOKEN = "wandb" # @param {type:"string"}
GITHUB_TOKEN = "github" # @param {type:"string"}

# Environment variables
runpod.api_key = userdata.get(RUNPOD_TOKEN)
WANDB_API_KEY = userdata.get(WANDB_TOKEN)
HF_TOKEN = userdata.get(HUGGING_FACE_TOKEN)
GITHUB_API_TOKEN = userdata.get(GITHUB_TOKEN)

# Make sure it's a valid YAML file
config = yaml.safe_load(yaml_config)

# Upload the YAML file to GitHub
gist_url = upload_gist(yaml_config, "config.yaml", GITHUB_API_TOKEN, f"{MODEL} - https://huggingface.co/{USERNAME}/{MODEL}")

# Summary
base_model = config.get('base_model', 'Unknown model')
dataset_info = []
datasets = config.get('datasets', [])
for dataset in datasets:
    path = dataset.get('path', 'Unknown path')
    dtype = dataset.get('type', 'Unknown type')
    dataset_info.append(f"{path} ({dtype})")
datasets_summary = ', '.join(dataset_info)

# Create a pod
keep_trying = True
try:
    while keep_trying:
        try:
            pod = runpod.create_pod(
                name=f"LazyAxolotl - {MODEL}",
                image_name="winglian/axolotl-cloud:main-latest",
                gpu_type_id=GPU,
                cloud_type=CLOUD_TYPE,
                gpu_count=NUMBER_OF_GPUS,
                volume_in_gb=0,
                container_disk_in_gb=CONTAINER_DISK,
                template_id="eul6o46pab",
                env={
                    "HF_TOKEN": HF_TOKEN,
                    "SCRIPT": SCRIPT,
                    "WANDB_API_KEY": WANDB_API_KEY,
                    "GIST_URL": gist_url,
                    "MODEL_NAME": MODEL,
                    "BASE_MODEL": config['base_model'],
                    "USERNAME": USERNAME,
                    "ZERO": ZERO,
                    "LLM_AUTOEVAL": LLM_AUTOEVAL,
                    "MODEL_ID": USERNAME + "/" + MODEL,
                    "BENCHMARK": "nous",
                    "REPO": "https://github.com/mlabonne/llm-autoeval.git",
                    "TRUST_REMOTE_CODE": True,
                    "GITHUB_API_TOKEN": GITHUB_API_TOKEN,
                    "DEBUG": DEBUG,
                }
            )
            print(f"This runs trains {base_model} on {datasets_summary}.")
            print("https://www.runpod.io/console/pods")
            keep_trying = False
        except QueryError as e:
            print(f"\033[31m⚠️ ERROR: The requested pod ({NUMBER_OF_GPUS}x {GPU} with {CONTAINER_DISK} GB) is not currently available. Trying again in 30 seconds...\033[39m")
            time.sleep(30)
except KeyboardInterrupt:
    print("KeyboardInterrupt detected, cleaning up before stopping...")
    delete_gist(GITHUB_API_TOKEN, gist_url.split('/')[4])

Uploaded Axolotl config as gist: https://gist.githubusercontent.com/mlabonne/9b239b771be74fadc4f0e10a769a9eff/raw/552c1dc5598de4db756a02634ea7ac061382fe94/config.yaml
This runs trains ai21labs/Jamba-v0.1 on mhenrichsen/alpaca_2k_test (alpaca).
https://www.runpod.io/console/pods
